use EfficientNet models for real-time applications using TensorFlow Lite. TensorFlow Lite is designed to optimize models for deployment on edge devices, providing smaller model sizes and faster inference times. EfficientNet models, especially the smaller variants like B0-B3, are well-suited for deployment using TensorFlow Lite on devices with limited computational resources, such as smartphones, Raspberry Pi, or IoT devices.

Steps to Use EfficientNet with TensorFlow Lite
Convert EfficientNet Model to TensorFlow Lite Format:

You need to convert the TensorFlow/Keras EfficientNet model to a .tflite format.
Use TensorFlow’s tf.lite.TFLiteConverter to convert the trained model.
Quantization:

Apply quantization techniques (e.g., post-training quantization) to further reduce the model size and improve inference speed.
Supported quantization types include float16, int8, and dynamic range.
Deploy on Edge Device:

Deploy the .tflite model on your target device, such as a Raspberry Pi or Android smartphone.
Use TensorFlow Lite’s interpreter to load and run inference.

In [6]:
import tensorflow 
import tflite 
print(tensorflow.__version__ , tflite.__version__)

2.17.0 2.10.0


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3

# Define input shape based on the dataset (e.g., 300x300 RGB images)
input_shape = (300, 300, 3)

# Load the EfficientNet-B3 model with custom settings
base_model = EfficientNetB3(
    include_top=False,            # Remove the default top layer to add custom layers
    weights='imagenet',           # Use pre-trained ImageNet weights
    input_shape=input_shape,      # Set input shape
    pooling='avg'                 # Apply global average pooling
)

# Freeze the base model layers to retain pre-trained features
base_model.trainable = False

# Create a new model on top of the base model
model = models.Sequential([
    base_model,                                    # EfficientNetB3 base model
    layers.Dense(256, activation='relu'),          # Custom fully connected layer for feature extraction
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),                           # Dropout layer for regularization
    layers.Dense(2, activation='softmax')          # Output layer with 2 classes (for 2 gun types)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',               # Use categorical cross-entropy for multi-class classification
    metrics=['accuracy']
)

# Summary of the custom model
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 1536)           │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,210,161 (42.76 MB)

 Trainable params: 426,626 (1.63 MB)

 Non-trainable params: 10,783,535 (41.14 MB)

In [1]:
# sample code to generate data generators
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3

# Define input shape based on your dataset (e.g., 300x300 RGB images)
input_shape = (300, 300, 3)

# Load the EfficientNet-B3 model with custom settings
base_model = EfficientNetB3(
    include_top=False,            # Remove the default top layer to add custom layers
    weights='imagenet',           # Use pre-trained ImageNet weights
    input_shape=input_shape,      # Set input shape
    pooling='avg'                 # Apply global average pooling
)

# Freeze the base model layers to retain pre-trained features
base_model.trainable = False

# Create a new model on top of the base model
model = models.Sequential([
    base_model,                                    # EfficientNetB3 base model
    layers.Dense(256, activation='relu'),          # Custom fully connected layer for feature extraction
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),                           # Dropout layer for regularization
    layers.Dense(2, activation='softmax')            # Output layer with 2 classes (handgun, big gun)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',               # Use categorical cross-entropy for multi-class classification
    metrics=['accuracy']
)

# Summary of the custom model
model.summary()

# Training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/train',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/home/research/Kilsar_Sentinal/EfficientNet data/splitted_data/val',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

2024-10-08 20:19:52.488564: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 20:19:52.500206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 20:19:52.511800: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 20:19:52.515284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 20:19:52.525567: I tensorflow/core/platform/cpu_feature_guar

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 1536)           │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,210,161 (42.76 MB)

 Trainable params: 426,626 (1.63 MB)

...

249/249 ━━━━━━━━━━━━━━━━━━━━ 65s 259ms/step - accuracy: 0.8068 - loss: 0.4867 - val_accuracy: 0.8090 - val_loss: 0.4753
Epoch 4/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 110us/step - accuracy: 0.8438 - loss: 0.4118 - val_accuracy: 0.8750 - val_loss: 0.3885
Epoch 5/10


2024-10-08 20:22:57.069000: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


249/249 ━━━━━━━━━━━━━━━━━━━━ 64s 258ms/step - accuracy: 0.8135 - loss: 0.4753 - val_accuracy: 0.8095 - val_loss: 0.4815
Epoch 6/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 117us/step - accuracy: 0.8750 - loss: 0.4214 - val_accuracy: 0.8125 - val_loss: 0.4684
Epoch 7/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 64s 258ms/step - accuracy: 0.8174 - loss: 0.4675 - val_accuracy: 0.8095 - val_loss: 0.4740
Epoch 8/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 110us/step - accuracy: 0.6875 - loss: 0.6429 - val_accuracy: 0.8125 - val_loss: 0.5187
Epoch 9/10


2024-10-08 20:25:05.906281: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


249/249 ━━━━━━━━━━━━━━━━━━━━ 64s 259ms/step - accuracy: 0.8105 - loss: 0.4765 - val_accuracy: 0.8090 - val_loss: 0.4753
Epoch 10/10
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 110us/step - accuracy: 0.9062 - loss: 0.3346 - val_accuracy: 0.8750 - val_loss: 0.3329


To save converted model

In [6]:
model.save('/home/research/Kilsar_Sentinal/suryansh/src/models/tf models/model-1.keras')

In [ ]:

converter = tf.lite.TFLiteConverter.from_saved_model('efficientnet_b0_saved_model')
# Optional: apply optimizations like quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Convert the model
tflite_model = converter.convert()

# Save the converted .tflite model
with open('efficientnet_b0.tflite', 'wb') as f:
    f.write(tflite_model)